In [1]:
import requests
import pandas as pd
import json
from cred import credentials
from request_api import get_genesis_api
from sql import get_data

In [38]:
data = pd.read_csv('all_merged_scaled_f.csv', index_col=0)
data

,year,state,age_avg,total_population,country,new_building,area_new_b,rooms_new_b,cinema,cinema_seat,...,scaled_age_avg,scaled_new_building,scaled_rooms_new_b,scaled_area_new_b,scaled_cinema,scaled_cinema_seat,scaled_museum,scaled_apartment,scaled_restaurant,score
0,2020,Baden-Württemberg,43.240650,11103043,Germany,26600,1402,186843,252.0,100970.0,...,0.275285,0.645766,0.618560,0.852220,0.220814,0.142081,0.784282,0.000000,0.556773,4.095780
1,2020,Bayern,43.488237,13140183,Germany,40427,1948,283342,284.0,125431.0,...,0.317052,0.926662,1.000000,1.000000,0.188775,0.189097,0.337564,0.132047,0.710754,4.801950
2,2020,Berlin,42.149704,3664088,Germany,3043,156,53307,95.0,48373.0,...,0.091246,0.000000,0.434919,0.289389,0.316358,0.569664,0.268316,0.520919,1.000000,3.490813
3,2020,Brandenburg,46.722180,2531071,Germany,8237,131,53611,64.0,24371.0,...,0.862605,1.000000,0.969237,0.351131,0.297385,0.197749,0.974735,0.445386,0.100073,5.198301
4,2020,Bremen,43.143783,680130,Germany,778,35,6223,12.0,11791.0,...,0.258944,0.129300,0.000000,0.349140,0.071390,1.000000,0.262221,0.466226,0.643403,3.180624
5,2020,Hamburg,41.608811,1852478,Germany,2221,139,31096,33.0,20595.0,...,0.000000,0.152006,0.615183,0.507667,0.076425,0.352711,0.000000,0.394256,0.988801,3.087049
6,2020,Hessen,43.518179,6293154,Germany,12237,494,97673,128.0,56589.0,...,0.322103,0.459598,0.513223,0.530957,0.151114,0.131490,0.386893,0.051506,0.645965,3.192848
7,2020,Mecklenburg-Vorpommern,46.846078,1610774,Germany,3454,68,24681,79.0,19778.0,...,0.883506,0.542035,0.497267,0.286971,1.000000,0.473557,1.000000,0.804288,0.707851,6.195475
8,2020,Niedersachsen,44.270348,8003421,Germany,22024,714,155134,177.0,69454.0,...,0.448991,0.792674,0.824418,0.603004,0.203634,0.098796,0.577593,0.209557,0.287010,4.045675
9,2020,Nordrhein-Westfalen,43.759486,17925570,Germany,30352,821,228644,273.0,171030.0,...,0.362810,0.355932,0.290452,0.311077,0.000000,0.188630,0.137301,0.219996,0.308391,2.174590


In [34]:
columns = ['age_avg', 'new_building', 'area_new_b', 'rooms_new_b', 'cinema', 'cinema_seat', 'museum',
       'apartment', 'restaurant']

default_weights = dict(zip(columns, [1] * 9))

def calc_score(weights=default_weights):
    data['w_score'] = [0] * 16
    for col in columns:
        data['w_score'] += data['scaled_%s' % col] * weights[col]
    return data[['state', 'w_score']].sort_values(by='w_score', ascending=False)

In [35]:
calc_score()

,state,w_score
7,Mecklenburg-Vorpommern,6.195475
3,Brandenburg,5.198301
1,Bayern,4.801950
14,Schleswig-Holstein,4.710946
10,Rheinland-Pfalz,4.338367
12,Sachsen,4.272058
0,Baden-Württemberg,4.095780
8,Niedersachsen,4.045675
13,Sachsen-Anhalt,3.730899
15,Thüringen,3.670636


In [37]:
calc_score({'age_avg':1, 'new_building':2, 'area_new_b':3, 'rooms_new_b':3, 'cinema':1, 'cinema_seat':1, 'museum':1,
       'apartment':3, 'restaurant':1})

,state,w_score
1,Bayern,9.992706
7,Mecklenburg-Vorpommern,9.914562
3,Brandenburg,9.729810
14,Schleswig-Holstein,9.347404
10,Rheinland-Pfalz,8.273698
8,Niedersachsen,8.112306
12,Sachsen,7.985348
0,Baden-Württemberg,7.683105
13,Sachsen-Anhalt,6.500458
5,Hamburg,6.273265
